In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader

In [2]:
t = AutoTokenizer.from_pretrained("./tasks/qa_feedback/model_outputs/t5-large-1k-train")

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': './tasks/qa_feedback/model_outputs/t5-large-1k-train'. Use `repo_type` argument if needed.

In [2]:
gt = ['"Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3. The track features American hip-hop artist will.i.am of The Black Eyed Peas. The song features Will.i.am of The Black Eyed Peas.', '"Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album #3. It features American hip-hop artist will.i.am of The Black Eyed Peas. The Script frontman Danny O\'Donoghue first met will.i.am while they were both judges for "The Voice UK".', '"Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album, Number 3. The song features American hip-hop artist will.i.am of The Black Eyed Peas. It was given its first radio play on Capital on 23 July 2012.', '"Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3". The song features American hip-hop artist will.i.am of The Black Eyed Peas. The song was given its first radio play on Capital on 23 July 2012.', 'In the New Testament, the golden rule is mentioned in verses 11 and 12. The rule is mentioned in Matthew 7:12, which is the twelfth verse of the seventh chapter of the Gospel of Matthew and is part of the Sermon on the Mount. In the King James Version of the Bible the text reads: The World English Bible translates the passage as: For a collection of other versions see BibleGateway Matthew 7:12.', 'Matthew 7:12 is the twelfth verse of the seventh chapter of the Gospel of Matthew in the New Testament and is part of the Sermon on the Mount. This well known verse presents what has become known as the Golden Rule.', 'The Golden Rule is mentioned in several places in the Bible. In Matthew 7:12, it is the twelfth verse of the seventh chapter of the Gospel of Matthew in the New Testament and is part of the Sermon on the Mount.', 'In the Bible, the Golden Rule is mentioned in several different verses. In Matthew 7:12, it is the twelfth verse of the seventh chapter of the Gospel of Matthew in the New Testament and is part of the Sermon on the Mount.', '"Hall of Fame" is a song by Irish pop rock band The Script. The track features American hip-hop artist will.i.am of The Black Eyed Peas.', '"Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3". The track features American hip-hop artist will.i.am of The Black Eyed Peas.', '"Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album, #3. The song features American hip-hop artist will.i.am of The Black Eyed Peas.', '"Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album #3. The track features American hip-hop artist will.i.am of The Black Eyed Peas. The song has been licensed for use in various media since its release.', 'Sie werden sowohl von der Frau als auch von der Frau als „Spouse" zugesprochen werden. Sie spricht Deutsch, Englisch, Französisch, Italienisch, Englisch, Japanisch und Deutsch und wird beim Telefonieren von Telefonen als „Berufsanzeiger" angezeigt.', 'Sie wird von einer anderen Person gespielt, deren Mutter ebenfalls gespielt wird. „Sie haben keine Angst, dass jemand ihre Stimme von anderen hören wird. Sie haben keine Angst, dass jemand ihre Stimme nicht abgeben wird.', "'Hall of Fame' is the lead single from Irish pop rock band The Script and features American hip-hop artist will.i.am of The Black Eyed Peas.", '"Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3. The track features American hip-hop artist will.i.am of The Black Eyed Peas. Affären beim Anfänger sind einfach anfällig für diesen Song, da affären sie sich vor ihrem Anfänger nicht immer so gut kennen.', '"Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3". The track features American hip-hop artist will.i.am of The Black Eyed Peas.', '"Hall of Fame" is the lead single from Irish pop rock band The Script. The song features American hip-hop artist will.i.am of The Black Eyed Peas. The song was given its first radio play on Capital on 23 July 2012. Written and co-produced by the band, the song is about following dreams and making an impact on the world.', '"Hall of Fame" is a song by Irish pop rock band The Script. The lead single from their third studio album is entitled "#3". The track features American hip-hop artist, will.i.am of The Black Eyed Peas.', '"Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3". The track features American hip-hop artist will.i.am of The Black Eyed Peas.', 'von ihnen zu erholen und zu vergleichen. Auch können Sie sich bei ihrem Gesprächspartner im Forum über das Thema oder über die Musik einstellen.', 'In ihr Zuhause in der Stadt wohnen, finden Sie unsere Stadtpläne und vieles mehr. Wir sind die Stadt der kleinen Leute, die ihren Lebensunterhalt mit Menschen, ihrer Heimat und ihrer Kultur verbinden.', 'A LOT von Menschen wohnt im Apartmenthaus "Stadthaus" in "Stadtwohnung" im Viertel "Stadtwohnung" in "Stadtwohnung". In den meisten wohnhäusern wohnt ein ganzes Apartmenthaus mit zwei Schlafzimmern.', 'Ihnen ein Stück der Geschichte beim ersten Auftritt auf dem "Hall of Fame" vorzuenthalten. Auf den anderen wird jedes Mal ein Stück der Geschichte erzählt.']

p = ['question: Who sings standing in the hall of fame? context: wikipage: Hall of Fame (song) text: Hall of Fame (song) "Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3". The track features American hip-hop artist will.i.am of The Black Eyed Peas. The track was given its first radio play on Capital on 23 July 2012. Written and co-produced by the band, the song is about following dreams and making an impact on the world. The song has been licensed for use in various media since its release. The Script frontman Danny O\'Donoghue first met will.i.am while they were both judges for "The Voice UK". O\'Donoghue told: "I played Will a few demos backstage early on in "The Voice". We got to Hall Of Fame and he immediately asked if he could have it for his new album, as it was exactly what he was looking for. I wasn’t having it, it was our lead single but I asked if he’d be on it. He agreed and we booked in time to record it. The problem is, he’s either late or completely flakes, so it took four attempts to get to record his part. Eventually we put it together in his hotel room in 15 minutes. It’s a classic duet, I do one line and he responds."', 'question: Who sings standing in the hall of fame? context: wikipage: Hall of Fame (song) text: Hall of Fame (song) "Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3". The track features American hip-hop artist will.i.am of The Black Eyed Peas. The track was given its first radio play on Capital on 23 July 2012. Written and co-produced by the band, the song is about following dreams and making an impact on the world. The song has been licensed for use in various media since its release. The Script frontman Danny O\'Donoghue first met will.i.am while they were both judges for "The Voice UK". O\'Donoghue told: "I played Will a few demos backstage early on in "The Voice". We got to Hall Of Fame and he immediately asked if he could have it for his new album, as it was exactly what he was looking for. I wasn’t having it, it was our lead single but I asked if he’d be on it. He agreed and we booked in time to record it. The problem is, he’s either late or completely flakes, so it took four attempts to get to record his part. Eventually we put it together in his hotel room in 15 minutes. It’s a classic duet, I do one line and he responds."', 'question: Who sings standing in the hall of fame? context: wikipage: Hall of Fame (song) text: Hall of Fame (song) "Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3". The track features American hip-hop artist will.i.am of The Black Eyed Peas. The track was given its first radio play on Capital on 23 July 2012. Written and co-produced by the band, the song is about following dreams and making an impact on the world. The song has been licensed for use in various media since its release. The Script frontman Danny O\'Donoghue first met will.i.am while they were both judges for "The Voice UK". O\'Donoghue told: "I played Will a few demos backstage early on in "The Voice". We got to Hall Of Fame and he immediately asked if he could have it for his new album, as it was exactly what he was looking for. I wasn’t having it, it was our lead single but I asked if he’d be on it. He agreed and we booked in time to record it. The problem is, he’s either late or completely flakes, so it took four attempts to get to record his part. Eventually we put it together in his hotel room in 15 minutes. It’s a classic duet, I do one line and he responds."', 'question: Who sings standing in the hall of fame? context: wikipage: Hall of Fame (song) text: Hall of Fame (song) "Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3". The track features American hip-hop artist will.i.am of The Black Eyed Peas. The track was given its first radio play on Capital on 23 July 2012. Written and co-produced by the band, the song is about following dreams and making an impact on the world. The song has been licensed for use in various media since its release. The Script frontman Danny O\'Donoghue first met will.i.am while they were both judges for "The Voice UK". O\'Donoghue told: "I played Will a few demos backstage early on in "The Voice". We got to Hall Of Fame and he immediately asked if he could have it for his new album, as it was exactly what he was looking for. I wasn’t having it, it was our lead single but I asked if he’d be on it. He agreed and we booked in time to record it. The problem is, he’s either late or completely flakes, so it took four attempts to get to record his part. Eventually we put it together in his hotel room in 15 minutes. It’s a classic duet, I do one line and he responds."', 'question: Where in the bible does it talk about the golden rule? context: wikipage: Matthew 7:12 text: Matthew 7:12 Matthew 7:12 is the twelfth verse of the seventh chapter of the Gospel of Matthew in the New Testament and is part of the Sermon on the Mount. This well known verse presents what has become known as the Golden Rule. In the King James Version of the Bible the text reads: The World English Bible translates the passage as: For a collection of other versions see BibleGateway Matthew 7:12 This verse is considered to be a summation of the entire sermon. Some editions append it to the end of -, and the rule does seem to be an expansion on the teaching about prayer in that section. However, the word therefore and the mention of the law and the prophets implies that this is a more far reaching teaching. Davies and Allison note that this is indicated by the mention of the law and the prophets, which links the verse back to Matthew 5:17, the start of the teaching on ethics. The verse is most closely linked with the teaching to "love thy enemies" in. In the Rule is present just after the teaching about enemies, making the link even more explicit. Luz notes that as well as summarizing the sermon, this rule also adapts it to normal life. While verses like seem incompatible with reality, the teachings in this verse can reasonably be attempted by all. | wikipage: Luke 10 text: Luke 10 Luke 10 is the tenth chapter of the Gospel of Luke in the New Testament of the Christian Bible. It records the sending of seventy disciples by Jesus, the famous parable about the Good Samaritan, and his visit to the house of Mary and Martha. The book containing this chapter is anonymous, but early Christian tradition uniformly affirmed that Luke composed this Gospel as well as Acts. The New King James Version organises this chapter as follows (with cross references to parallel sections in the other gospels): Theologian Heinrich Meyer calls this section the "Narrative of the Seventy" and links it to the earlier account of the sending out of advance messengers in. This parable is recounted only in this chapter of the New Testament. A lawyer or \'expert in the law\' asked Jesus to explain who his \'neighbour\' is, referring to the ordinance of Leviticus 19:18: In response, Jesus told a story of a traveller (who may or may not have been a Jew ) who is beaten, robbed, and left half dead along the road. First a priest and then a Levite come by, but both avoid the man. Finally, a Samaritan comes by. Samaritans and Jews generally despised each other, but the Samaritan helps the injured man. Portraying a Samaritan in a positive light would have come as a shock to Jesus\'s audience.', 'question: Where in the bible does it talk about the golden rule? context: wikipage: Matthew 7:12 text: Matthew 7:12 Matthew 7:12 is the twelfth verse of the seventh chapter of the Gospel of Matthew in the New Testament and is part of the Sermon on the Mount. This well known verse presents what has become known as the Golden Rule. In the King James Version of the Bible the text reads: The World English Bible translates the passage as: For a collection of other versions see BibleGateway Matthew 7:12 This verse is considered to be a summation of the entire sermon. Some editions append it to the end of -, and the rule does seem to be an expansion on the teaching about prayer in that section. However, the word therefore and the mention of the law and the prophets implies that this is a more far reaching teaching. Davies and Allison note that this is indicated by the mention of the law and the prophets, which links the verse back to Matthew 5:17, the start of the teaching on ethics. The verse is most closely linked with the teaching to "love thy enemies" in. In the Rule is present just after the teaching about enemies, making the link even more explicit. Luz notes that as well as summarizing the sermon, this rule also adapts it to normal life. While verses like seem incompatible with reality, the teachings in this verse can reasonably be attempted by all. | wikipage: Luke 10 text: Luke 10 Luke 10 is the tenth chapter of the Gospel of Luke in the New Testament of the Christian Bible. It records the sending of seventy disciples by Jesus, the famous parable about the Good Samaritan, and his visit to the house of Mary and Martha. The book containing this chapter is anonymous, but early Christian tradition uniformly affirmed that Luke composed this Gospel as well as Acts. The New King James Version organises this chapter as follows (with cross references to parallel sections in the other gospels): Theologian Heinrich Meyer calls this section the "Narrative of the Seventy" and links it to the earlier account of the sending out of advance messengers in. This parable is recounted only in this chapter of the New Testament. A lawyer or \'expert in the law\' asked Jesus to explain who his \'neighbour\' is, referring to the ordinance of Leviticus 19:18: In response, Jesus told a story of a traveller (who may or may not have been a Jew ) who is beaten, robbed, and left half dead along the road. First a priest and then a Levite come by, but both avoid the man. Finally, a Samaritan comes by. Samaritans and Jews generally despised each other, but the Samaritan helps the injured man. Portraying a Samaritan in a positive light would have come as a shock to Jesus\'s audience.', 'question: Where in the bible does it talk about the golden rule? context: wikipage: Matthew 7:12 text: Matthew 7:12 Matthew 7:12 is the twelfth verse of the seventh chapter of the Gospel of Matthew in the New Testament and is part of the Sermon on the Mount. This well known verse presents what has become known as the Golden Rule. In the King James Version of the Bible the text reads: The World English Bible translates the passage as: For a collection of other versions see BibleGateway Matthew 7:12 This verse is considered to be a summation of the entire sermon. Some editions append it to the end of -, and the rule does seem to be an expansion on the teaching about prayer in that section. However, the word therefore and the mention of the law and the prophets implies that this is a more far reaching teaching. Davies and Allison note that this is indicated by the mention of the law and the prophets, which links the verse back to Matthew 5:17, the start of the teaching on ethics. The verse is most closely linked with the teaching to "love thy enemies" in. In the Rule is present just after the teaching about enemies, making the link even more explicit. Luz notes that as well as summarizing the sermon, this rule also adapts it to normal life. While verses like seem incompatible with reality, the teachings in this verse can reasonably be attempted by all. | wikipage: Luke 10 text: Luke 10 Luke 10 is the tenth chapter of the Gospel of Luke in the New Testament of the Christian Bible. It records the sending of seventy disciples by Jesus, the famous parable about the Good Samaritan, and his visit to the house of Mary and Martha. The book containing this chapter is anonymous, but early Christian tradition uniformly affirmed that Luke composed this Gospel as well as Acts. The New King James Version organises this chapter as follows (with cross references to parallel sections in the other gospels): Theologian Heinrich Meyer calls this section the "Narrative of the Seventy" and links it to the earlier account of the sending out of advance messengers in. This parable is recounted only in this chapter of the New Testament. A lawyer or \'expert in the law\' asked Jesus to explain who his \'neighbour\' is, referring to the ordinance of Leviticus 19:18: In response, Jesus told a story of a traveller (who may or may not have been a Jew ) who is beaten, robbed, and left half dead along the road. First a priest and then a Levite come by, but both avoid the man. Finally, a Samaritan comes by. Samaritans and Jews generally despised each other, but the Samaritan helps the injured man. Portraying a Samaritan in a positive light would have come as a shock to Jesus\'s audience.', 'question: Where in the bible does it talk about the golden rule? context: wikipage: Matthew 7:12 text: Matthew 7:12 Matthew 7:12 is the twelfth verse of the seventh chapter of the Gospel of Matthew in the New Testament and is part of the Sermon on the Mount. This well known verse presents what has become known as the Golden Rule. In the King James Version of the Bible the text reads: The World English Bible translates the passage as: For a collection of other versions see BibleGateway Matthew 7:12 This verse is considered to be a summation of the entire sermon. Some editions append it to the end of -, and the rule does seem to be an expansion on the teaching about prayer in that section. However, the word therefore and the mention of the law and the prophets implies that this is a more far reaching teaching. Davies and Allison note that this is indicated by the mention of the law and the prophets, which links the verse back to Matthew 5:17, the start of the teaching on ethics. The verse is most closely linked with the teaching to "love thy enemies" in. In the Rule is present just after the teaching about enemies, making the link even more explicit. Luz notes that as well as summarizing the sermon, this rule also adapts it to normal life. While verses like seem incompatible with reality, the teachings in this verse can reasonably be attempted by all. | wikipage: Luke 10 text: Luke 10 Luke 10 is the tenth chapter of the Gospel of Luke in the New Testament of the Christian Bible. It records the sending of seventy disciples by Jesus, the famous parable about the Good Samaritan, and his visit to the house of Mary and Martha. The book containing this chapter is anonymous, but early Christian tradition uniformly affirmed that Luke composed this Gospel as well as Acts. The New King James Version organises this chapter as follows (with cross references to parallel sections in the other gospels): Theologian Heinrich Meyer calls this section the "Narrative of the Seventy" and links it to the earlier account of the sending out of advance messengers in. This parable is recounted only in this chapter of the New Testament. A lawyer or \'expert in the law\' asked Jesus to explain who his \'neighbour\' is, referring to the ordinance of Leviticus 19:18: In response, Jesus told a story of a traveller (who may or may not have been a Jew ) who is beaten, robbed, and left half dead along the road. First a priest and then a Levite come by, but both avoid the man. Finally, a Samaritan comes by. Samaritans and Jews generally despised each other, but the Samaritan helps the injured man. Portraying a Samaritan in a positive light would have come as a shock to Jesus\'s audience.', 'question: Who sings standing in the hall of fame? context: wikipage: Hall of Fame (song) text: Hall of Fame (song) "Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3". The track features American hip-hop artist will.i.am of The Black Eyed Peas. The track was given its first radio play on Capital on 23 July 2012. Written and co-produced by the band, the song is about following dreams and making an impact on the world. The song has been licensed for use in various media since its release. The Script frontman Danny O\'Donoghue first met will.i.am while they were both judges for "The Voice UK". O\'Donoghue told: "I played Will a few demos backstage early on in "The Voice". We got to Hall Of Fame and he immediately asked if he could have it for his new album, as it was exactly what he was looking for. I wasn’t having it, it was our lead single but I asked if he’d be on it. He agreed and we booked in time to record it. The problem is, he’s either late or completely flakes, so it took four attempts to get to record his part. Eventually we put it together in his hotel room in 15 minutes. It’s a classic duet, I do one line and he responds."', 'question: Who sings standing in the hall of fame? context: wikipage: Hall of Fame (song) text: Hall of Fame (song) "Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3". The track features American hip-hop artist will.i.am of The Black Eyed Peas. The track was given its first radio play on Capital on 23 July 2012. Written and co-produced by the band, the song is about following dreams and making an impact on the world. The song has been licensed for use in various media since its release. The Script frontman Danny O\'Donoghue first met will.i.am while they were both judges for "The Voice UK". O\'Donoghue told: "I played Will a few demos backstage early on in "The Voice". We got to Hall Of Fame and he immediately asked if he could have it for his new album, as it was exactly what he was looking for. I wasn’t having it, it was our lead single but I asked if he’d be on it. He agreed and we booked in time to record it. The problem is, he’s either late or completely flakes, so it took four attempts to get to record his part. Eventually we put it together in his hotel room in 15 minutes. It’s a classic duet, I do one line and he responds."', 'question: Who sings standing in the hall of fame? context: wikipage: Hall of Fame (song) text: Hall of Fame (song) "Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3". The track features American hip-hop artist will.i.am of The Black Eyed Peas. The track was given its first radio play on Capital on 23 July 2012. Written and co-produced by the band, the song is about following dreams and making an impact on the world. The song has been licensed for use in various media since its release. The Script frontman Danny O\'Donoghue first met will.i.am while they were both judges for "The Voice UK". O\'Donoghue told: "I played Will a few demos backstage early on in "The Voice". We got to Hall Of Fame and he immediately asked if he could have it for his new album, as it was exactly what he was looking for. I wasn’t having it, it was our lead single but I asked if he’d be on it. He agreed and we booked in time to record it. The problem is, he’s either late or completely flakes, so it took four attempts to get to record his part. Eventually we put it together in his hotel room in 15 minutes. It’s a classic duet, I do one line and he responds."', 'question: Who sings standing in the hall of fame? context: wikipage: Hall of Fame (song) text: Hall of Fame (song) "Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3". The track features American hip-hop artist will.i.am of The Black Eyed Peas. The track was given its first radio play on Capital on 23 July 2012. Written and co-produced by the band, the song is about following dreams and making an impact on the world. The song has been licensed for use in various media since its release. The Script frontman Danny O\'Donoghue first met will.i.am while they were both judges for "The Voice UK". O\'Donoghue told: "I played Will a few demos backstage early on in "The Voice". We got to Hall Of Fame and he immediately asked if he could have it for his new album, as it was exactly what he was looking for. I wasn’t having it, it was our lead single but I asked if he’d be on it. He agreed and we booked in time to record it. The problem is, he’s either late or completely flakes, so it took four attempts to get to record his part. Eventually we put it together in his hotel room in 15 minutes. It’s a classic duet, I do one line and he responds."', 'question: Where in the bible does it talk about the golden rule? context: wikipage: Matthew 7:12 text: Matthew 7:12 Matthew 7:12 is the twelfth verse of the seventh chapter of the Gospel of Matthew in the New Testament and is part of the Sermon on the Mount. This well known verse presents what has become known as the Golden Rule. In the King James Version of the Bible the text reads: The World English Bible translates the passage as: For a collection of other versions see BibleGateway Matthew 7:12 This verse is considered to be a summation of the entire sermon. Some editions append it to the end of -, and the rule does seem to be an expansion on the teaching about prayer in that section. However, the word therefore and the mention of the law and the prophets implies that this is a more far reaching teaching. Davies and Allison note that this is indicated by the mention of the law and the prophets, which links the verse back to Matthew 5:17, the start of the teaching on ethics. The verse is most closely linked with the teaching to "love thy enemies" in. In the Rule is present just after the teaching about enemies, making the link even more explicit. Luz notes that as well as summarizing the sermon, this rule also adapts it to normal life. While verses like seem incompatible with reality, the teachings in this verse can reasonably be attempted by all. | wikipage: Luke 10 text: Luke 10 Luke 10 is the tenth chapter of the Gospel of Luke in the New Testament of the Christian Bible. It records the sending of seventy disciples by Jesus, the famous parable about the Good Samaritan, and his visit to the house of Mary and Martha. The book containing this chapter is anonymous, but early Christian tradition uniformly affirmed that Luke composed this Gospel as well as Acts. The New King James Version organises this chapter as follows (with cross references to parallel sections in the other gospels): Theologian Heinrich Meyer calls this section the "Narrative of the Seventy" and links it to the earlier account of the sending out of advance messengers in. This parable is recounted only in this chapter of the New Testament. A lawyer or \'expert in the law\' asked Jesus to explain who his \'neighbour\' is, referring to the ordinance of Leviticus 19:18: In response, Jesus told a story of a traveller (who may or may not have been a Jew ) who is beaten, robbed, and left half dead along the road. First a priest and then a Levite come by, but both avoid the man. Finally, a Samaritan comes by. Samaritans and Jews generally despised each other, but the Samaritan helps the injured man. Portraying a Samaritan in a positive light would have come as a shock to Jesus\'s audience.', 'question: Where in the bible does it talk about the golden rule? context: wikipage: Matthew 7:12 text: Matthew 7:12 Matthew 7:12 is the twelfth verse of the seventh chapter of the Gospel of Matthew in the New Testament and is part of the Sermon on the Mount. This well known verse presents what has become known as the Golden Rule. In the King James Version of the Bible the text reads: The World English Bible translates the passage as: For a collection of other versions see BibleGateway Matthew 7:12 This verse is considered to be a summation of the entire sermon. Some editions append it to the end of -, and the rule does seem to be an expansion on the teaching about prayer in that section. However, the word therefore and the mention of the law and the prophets implies that this is a more far reaching teaching. Davies and Allison note that this is indicated by the mention of the law and the prophets, which links the verse back to Matthew 5:17, the start of the teaching on ethics. The verse is most closely linked with the teaching to "love thy enemies" in. In the Rule is present just after the teaching about enemies, making the link even more explicit. Luz notes that as well as summarizing the sermon, this rule also adapts it to normal life. While verses like seem incompatible with reality, the teachings in this verse can reasonably be attempted by all. | wikipage: Luke 10 text: Luke 10 Luke 10 is the tenth chapter of the Gospel of Luke in the New Testament of the Christian Bible. It records the sending of seventy disciples by Jesus, the famous parable about the Good Samaritan, and his visit to the house of Mary and Martha. The book containing this chapter is anonymous, but early Christian tradition uniformly affirmed that Luke composed this Gospel as well as Acts. The New King James Version organises this chapter as follows (with cross references to parallel sections in the other gospels): Theologian Heinrich Meyer calls this section the "Narrative of the Seventy" and links it to the earlier account of the sending out of advance messengers in. This parable is recounted only in this chapter of the New Testament. A lawyer or \'expert in the law\' asked Jesus to explain who his \'neighbour\' is, referring to the ordinance of Leviticus 19:18: In response, Jesus told a story of a traveller (who may or may not have been a Jew ) who is beaten, robbed, and left half dead along the road. First a priest and then a Levite come by, but both avoid the man. Finally, a Samaritan comes by. Samaritans and Jews generally despised each other, but the Samaritan helps the injured man. Portraying a Samaritan in a positive light would have come as a shock to Jesus\'s audience.', 'question: Where in the bible does it talk about the golden rule? context: wikipage: Matthew 7:12 text: Matthew 7:12 Matthew 7:12 is the twelfth verse of the seventh chapter of the Gospel of Matthew in the New Testament and is part of the Sermon on the Mount. This well known verse presents what has become known as the Golden Rule. In the King James Version of the Bible the text reads: The World English Bible translates the passage as: For a collection of other versions see BibleGateway Matthew 7:12 This verse is considered to be a summation of the entire sermon. Some editions append it to the end of -, and the rule does seem to be an expansion on the teaching about prayer in that section. However, the word therefore and the mention of the law and the prophets implies that this is a more far reaching teaching. Davies and Allison note that this is indicated by the mention of the law and the prophets, which links the verse back to Matthew 5:17, the start of the teaching on ethics. The verse is most closely linked with the teaching to "love thy enemies" in. In the Rule is present just after the teaching about enemies, making the link even more explicit. Luz notes that as well as summarizing the sermon, this rule also adapts it to normal life. While verses like seem incompatible with reality, the teachings in this verse can reasonably be attempted by all. | wikipage: Luke 10 text: Luke 10 Luke 10 is the tenth chapter of the Gospel of Luke in the New Testament of the Christian Bible. It records the sending of seventy disciples by Jesus, the famous parable about the Good Samaritan, and his visit to the house of Mary and Martha. The book containing this chapter is anonymous, but early Christian tradition uniformly affirmed that Luke composed this Gospel as well as Acts. The New King James Version organises this chapter as follows (with cross references to parallel sections in the other gospels): Theologian Heinrich Meyer calls this section the "Narrative of the Seventy" and links it to the earlier account of the sending out of advance messengers in. This parable is recounted only in this chapter of the New Testament. A lawyer or \'expert in the law\' asked Jesus to explain who his \'neighbour\' is, referring to the ordinance of Leviticus 19:18: In response, Jesus told a story of a traveller (who may or may not have been a Jew ) who is beaten, robbed, and left half dead along the road. First a priest and then a Levite come by, but both avoid the man. Finally, a Samaritan comes by. Samaritans and Jews generally despised each other, but the Samaritan helps the injured man. Portraying a Samaritan in a positive light would have come as a shock to Jesus\'s audience.', 'question: Where in the bible does it talk about the golden rule? context: wikipage: Matthew 7:12 text: Matthew 7:12 Matthew 7:12 is the twelfth verse of the seventh chapter of the Gospel of Matthew in the New Testament and is part of the Sermon on the Mount. This well known verse presents what has become known as the Golden Rule. In the King James Version of the Bible the text reads: The World English Bible translates the passage as: For a collection of other versions see BibleGateway Matthew 7:12 This verse is considered to be a summation of the entire sermon. Some editions append it to the end of -, and the rule does seem to be an expansion on the teaching about prayer in that section. However, the word therefore and the mention of the law and the prophets implies that this is a more far reaching teaching. Davies and Allison note that this is indicated by the mention of the law and the prophets, which links the verse back to Matthew 5:17, the start of the teaching on ethics. The verse is most closely linked with the teaching to "love thy enemies" in. In the Rule is present just after the teaching about enemies, making the link even more explicit. Luz notes that as well as summarizing the sermon, this rule also adapts it to normal life. While verses like seem incompatible with reality, the teachings in this verse can reasonably be attempted by all. | wikipage: Luke 10 text: Luke 10 Luke 10 is the tenth chapter of the Gospel of Luke in the New Testament of the Christian Bible. It records the sending of seventy disciples by Jesus, the famous parable about the Good Samaritan, and his visit to the house of Mary and Martha. The book containing this chapter is anonymous, but early Christian tradition uniformly affirmed that Luke composed this Gospel as well as Acts. The New King James Version organises this chapter as follows (with cross references to parallel sections in the other gospels): Theologian Heinrich Meyer calls this section the "Narrative of the Seventy" and links it to the earlier account of the sending out of advance messengers in. This parable is recounted only in this chapter of the New Testament. A lawyer or \'expert in the law\' asked Jesus to explain who his \'neighbour\' is, referring to the ordinance of Leviticus 19:18: In response, Jesus told a story of a traveller (who may or may not have been a Jew ) who is beaten, robbed, and left half dead along the road. First a priest and then a Levite come by, but both avoid the man. Finally, a Samaritan comes by. Samaritans and Jews generally despised each other, but the Samaritan helps the injured man. Portraying a Samaritan in a positive light would have come as a shock to Jesus\'s audience.', 'question: Who sings standing in the hall of fame? context: wikipage: Hall of Fame (song) text: Hall of Fame (song) "Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3". The track features American hip-hop artist will.i.am of The Black Eyed Peas. The track was given its first radio play on Capital on 23 July 2012. Written and co-produced by the band, the song is about following dreams and making an impact on the world. The song has been licensed for use in various media since its release. The Script frontman Danny O\'Donoghue first met will.i.am while they were both judges for "The Voice UK". O\'Donoghue told: "I played Will a few demos backstage early on in "The Voice". We got to Hall Of Fame and he immediately asked if he could have it for his new album, as it was exactly what he was looking for. I wasn’t having it, it was our lead single but I asked if he’d be on it. He agreed and we booked in time to record it. The problem is, he’s either late or completely flakes, so it took four attempts to get to record his part. Eventually we put it together in his hotel room in 15 minutes. It’s a classic duet, I do one line and he responds."', 'question: Who sings standing in the hall of fame? context: wikipage: Hall of Fame (song) text: Hall of Fame (song) "Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3". The track features American hip-hop artist will.i.am of The Black Eyed Peas. The track was given its first radio play on Capital on 23 July 2012. Written and co-produced by the band, the song is about following dreams and making an impact on the world. The song has been licensed for use in various media since its release. The Script frontman Danny O\'Donoghue first met will.i.am while they were both judges for "The Voice UK". O\'Donoghue told: "I played Will a few demos backstage early on in "The Voice". We got to Hall Of Fame and he immediately asked if he could have it for his new album, as it was exactly what he was looking for. I wasn’t having it, it was our lead single but I asked if he’d be on it. He agreed and we booked in time to record it. The problem is, he’s either late or completely flakes, so it took four attempts to get to record his part. Eventually we put it together in his hotel room in 15 minutes. It’s a classic duet, I do one line and he responds."', 'question: Who sings standing in the hall of fame? context: wikipage: Hall of Fame (song) text: Hall of Fame (song) "Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3". The track features American hip-hop artist will.i.am of The Black Eyed Peas. The track was given its first radio play on Capital on 23 July 2012. Written and co-produced by the band, the song is about following dreams and making an impact on the world. The song has been licensed for use in various media since its release. The Script frontman Danny O\'Donoghue first met will.i.am while they were both judges for "The Voice UK". O\'Donoghue told: "I played Will a few demos backstage early on in "The Voice". We got to Hall Of Fame and he immediately asked if he could have it for his new album, as it was exactly what he was looking for. I wasn’t having it, it was our lead single but I asked if he’d be on it. He agreed and we booked in time to record it. The problem is, he’s either late or completely flakes, so it took four attempts to get to record his part. Eventually we put it together in his hotel room in 15 minutes. It’s a classic duet, I do one line and he responds."', 'question: Who sings standing in the hall of fame? context: wikipage: Hall of Fame (song) text: Hall of Fame (song) "Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3". The track features American hip-hop artist will.i.am of The Black Eyed Peas. The track was given its first radio play on Capital on 23 July 2012. Written and co-produced by the band, the song is about following dreams and making an impact on the world. The song has been licensed for use in various media since its release. The Script frontman Danny O\'Donoghue first met will.i.am while they were both judges for "The Voice UK". O\'Donoghue told: "I played Will a few demos backstage early on in "The Voice". We got to Hall Of Fame and he immediately asked if he could have it for his new album, as it was exactly what he was looking for. I wasn’t having it, it was our lead single but I asked if he’d be on it. He agreed and we booked in time to record it. The problem is, he’s either late or completely flakes, so it took four attempts to get to record his part. Eventually we put it together in his hotel room in 15 minutes. It’s a classic duet, I do one line and he responds."', 'question: Where in the bible does it talk about the golden rule? context: wikipage: Matthew 7:12 text: Matthew 7:12 Matthew 7:12 is the twelfth verse of the seventh chapter of the Gospel of Matthew in the New Testament and is part of the Sermon on the Mount. This well known verse presents what has become known as the Golden Rule. In the King James Version of the Bible the text reads: The World English Bible translates the passage as: For a collection of other versions see BibleGateway Matthew 7:12 This verse is considered to be a summation of the entire sermon. Some editions append it to the end of -, and the rule does seem to be an expansion on the teaching about prayer in that section. However, the word therefore and the mention of the law and the prophets implies that this is a more far reaching teaching. Davies and Allison note that this is indicated by the mention of the law and the prophets, which links the verse back to Matthew 5:17, the start of the teaching on ethics. The verse is most closely linked with the teaching to "love thy enemies" in. In the Rule is present just after the teaching about enemies, making the link even more explicit. Luz notes that as well as summarizing the sermon, this rule also adapts it to normal life. While verses like seem incompatible with reality, the teachings in this verse can reasonably be attempted by all. | wikipage: Luke 10 text: Luke 10 Luke 10 is the tenth chapter of the Gospel of Luke in the New Testament of the Christian Bible. It records the sending of seventy disciples by Jesus, the famous parable about the Good Samaritan, and his visit to the house of Mary and Martha. The book containing this chapter is anonymous, but early Christian tradition uniformly affirmed that Luke composed this Gospel as well as Acts. The New King James Version organises this chapter as follows (with cross references to parallel sections in the other gospels): Theologian Heinrich Meyer calls this section the "Narrative of the Seventy" and links it to the earlier account of the sending out of advance messengers in. This parable is recounted only in this chapter of the New Testament. A lawyer or \'expert in the law\' asked Jesus to explain who his \'neighbour\' is, referring to the ordinance of Leviticus 19:18: In response, Jesus told a story of a traveller (who may or may not have been a Jew ) who is beaten, robbed, and left half dead along the road. First a priest and then a Levite come by, but both avoid the man. Finally, a Samaritan comes by. Samaritans and Jews generally despised each other, but the Samaritan helps the injured man. Portraying a Samaritan in a positive light would have come as a shock to Jesus\'s audience.', 'question: Where in the bible does it talk about the golden rule? context: wikipage: Matthew 7:12 text: Matthew 7:12 Matthew 7:12 is the twelfth verse of the seventh chapter of the Gospel of Matthew in the New Testament and is part of the Sermon on the Mount. This well known verse presents what has become known as the Golden Rule. In the King James Version of the Bible the text reads: The World English Bible translates the passage as: For a collection of other versions see BibleGateway Matthew 7:12 This verse is considered to be a summation of the entire sermon. Some editions append it to the end of -, and the rule does seem to be an expansion on the teaching about prayer in that section. However, the word therefore and the mention of the law and the prophets implies that this is a more far reaching teaching. Davies and Allison note that this is indicated by the mention of the law and the prophets, which links the verse back to Matthew 5:17, the start of the teaching on ethics. The verse is most closely linked with the teaching to "love thy enemies" in. In the Rule is present just after the teaching about enemies, making the link even more explicit. Luz notes that as well as summarizing the sermon, this rule also adapts it to normal life. While verses like seem incompatible with reality, the teachings in this verse can reasonably be attempted by all. | wikipage: Luke 10 text: Luke 10 Luke 10 is the tenth chapter of the Gospel of Luke in the New Testament of the Christian Bible. It records the sending of seventy disciples by Jesus, the famous parable about the Good Samaritan, and his visit to the house of Mary and Martha. The book containing this chapter is anonymous, but early Christian tradition uniformly affirmed that Luke composed this Gospel as well as Acts. The New King James Version organises this chapter as follows (with cross references to parallel sections in the other gospels): Theologian Heinrich Meyer calls this section the "Narrative of the Seventy" and links it to the earlier account of the sending out of advance messengers in. This parable is recounted only in this chapter of the New Testament. A lawyer or \'expert in the law\' asked Jesus to explain who his \'neighbour\' is, referring to the ordinance of Leviticus 19:18: In response, Jesus told a story of a traveller (who may or may not have been a Jew ) who is beaten, robbed, and left half dead along the road. First a priest and then a Levite come by, but both avoid the man. Finally, a Samaritan comes by. Samaritans and Jews generally despised each other, but the Samaritan helps the injured man. Portraying a Samaritan in a positive light would have come as a shock to Jesus\'s audience.', 'question: Where in the bible does it talk about the golden rule? context: wikipage: Matthew 7:12 text: Matthew 7:12 Matthew 7:12 is the twelfth verse of the seventh chapter of the Gospel of Matthew in the New Testament and is part of the Sermon on the Mount. This well known verse presents what has become known as the Golden Rule. In the King James Version of the Bible the text reads: The World English Bible translates the passage as: For a collection of other versions see BibleGateway Matthew 7:12 This verse is considered to be a summation of the entire sermon. Some editions append it to the end of -, and the rule does seem to be an expansion on the teaching about prayer in that section. However, the word therefore and the mention of the law and the prophets implies that this is a more far reaching teaching. Davies and Allison note that this is indicated by the mention of the law and the prophets, which links the verse back to Matthew 5:17, the start of the teaching on ethics. The verse is most closely linked with the teaching to "love thy enemies" in. In the Rule is present just after the teaching about enemies, making the link even more explicit. Luz notes that as well as summarizing the sermon, this rule also adapts it to normal life. While verses like seem incompatible with reality, the teachings in this verse can reasonably be attempted by all. | wikipage: Luke 10 text: Luke 10 Luke 10 is the tenth chapter of the Gospel of Luke in the New Testament of the Christian Bible. It records the sending of seventy disciples by Jesus, the famous parable about the Good Samaritan, and his visit to the house of Mary and Martha. The book containing this chapter is anonymous, but early Christian tradition uniformly affirmed that Luke composed this Gospel as well as Acts. The New King James Version organises this chapter as follows (with cross references to parallel sections in the other gospels): Theologian Heinrich Meyer calls this section the "Narrative of the Seventy" and links it to the earlier account of the sending out of advance messengers in. This parable is recounted only in this chapter of the New Testament. A lawyer or \'expert in the law\' asked Jesus to explain who his \'neighbour\' is, referring to the ordinance of Leviticus 19:18: In response, Jesus told a story of a traveller (who may or may not have been a Jew ) who is beaten, robbed, and left half dead along the road. First a priest and then a Levite come by, but both avoid the man. Finally, a Samaritan comes by. Samaritans and Jews generally despised each other, but the Samaritan helps the injured man. Portraying a Samaritan in a positive light would have come as a shock to Jesus\'s audience.', 'question: Where in the bible does it talk about the golden rule? context: wikipage: Matthew 7:12 text: Matthew 7:12 Matthew 7:12 is the twelfth verse of the seventh chapter of the Gospel of Matthew in the New Testament and is part of the Sermon on the Mount. This well known verse presents what has become known as the Golden Rule. In the King James Version of the Bible the text reads: The World English Bible translates the passage as: For a collection of other versions see BibleGateway Matthew 7:12 This verse is considered to be a summation of the entire sermon. Some editions append it to the end of -, and the rule does seem to be an expansion on the teaching about prayer in that section. However, the word therefore and the mention of the law and the prophets implies that this is a more far reaching teaching. Davies and Allison note that this is indicated by the mention of the law and the prophets, which links the verse back to Matthew 5:17, the start of the teaching on ethics. The verse is most closely linked with the teaching to "love thy enemies" in. In the Rule is present just after the teaching about enemies, making the link even more explicit. Luz notes that as well as summarizing the sermon, this rule also adapts it to normal life. While verses like seem incompatible with reality, the teachings in this verse can reasonably be attempted by all. | wikipage: Luke 10 text: Luke 10 Luke 10 is the tenth chapter of the Gospel of Luke in the New Testament of the Christian Bible. It records the sending of seventy disciples by Jesus, the famous parable about the Good Samaritan, and his visit to the house of Mary and Martha. The book containing this chapter is anonymous, but early Christian tradition uniformly affirmed that Luke composed this Gospel as well as Acts. The New King James Version organises this chapter as follows (with cross references to parallel sections in the other gospels): Theologian Heinrich Meyer calls this section the "Narrative of the Seventy" and links it to the earlier account of the sending out of advance messengers in. This parable is recounted only in this chapter of the New Testament. A lawyer or \'expert in the law\' asked Jesus to explain who his \'neighbour\' is, referring to the ordinance of Leviticus 19:18: In response, Jesus told a story of a traveller (who may or may not have been a Jew ) who is beaten, robbed, and left half dead along the road. First a priest and then a Levite come by, but both avoid the man. Finally, a Samaritan comes by. Samaritans and Jews generally despised each other, but the Samaritan helps the injured man. Portraying a Samaritan in a positive light would have come as a shock to Jesus\'s audience.']

for pmp,res in zip(p,gt):
    print(f"Prompt:\n{pmp}\n\nGenerated_Text:\n{res}\n\n---------------------\n")

Prompt:
question: Who sings standing in the hall of fame? context: wikipage: Hall of Fame (song) text: Hall of Fame (song) "Hall of Fame" is a song by Irish pop rock band The Script. It is the lead single from their third studio album "#3". The track features American hip-hop artist will.i.am of The Black Eyed Peas. The track was given its first radio play on Capital on 23 July 2012. Written and co-produced by the band, the song is about following dreams and making an impact on the world. The song has been licensed for use in various media since its release. The Script frontman Danny O'Donoghue first met will.i.am while they were both judges for "The Voice UK". O'Donoghue told: "I played Will a few demos backstage early on in "The Voice". We got to Hall Of Fame and he immediately asked if he could have it for his new album, as it was exactly what he was looking for. I wasn’t having it, it was our lead single but I asked if he’d be on it. He agreed and we booked in time to record it. Th

In [7]:
for i in range(len(gt)):
    for j in range(len(gt)):
        if gt[i]==gt[j] and i!=j:
            print((i//8,i%8),(j//8,j%8))

(1, 1) (2, 0)
(1, 1) (2, 3)
(2, 0) (1, 1)
(2, 0) (2, 3)
(2, 3) (1, 1)
(2, 3) (2, 0)


In [9]:
rew_raw = [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.20838826583994596], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.48472406252770384], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.5171402280765716], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.3460630169150571], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, -0.41466123468855576], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, -0.1825296447662562], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, -0.24227857997257224], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.2, -0.15537933460077816], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, -0.1640484351815827], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.13229579908635067], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.23818141248139188], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.3879326107097781], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, -0.49500910208704707], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, -0.4318138744779004], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, -0.15485378991560453], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.006803445917395851], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.13229579908635067], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.1012898000172167], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.06867053023975417], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.13229579908635067], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, -0.5109164403342931], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, -0.31604580832698587], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, -0.4975545395217781], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, -0.29664034181066007]]

rew = [sum(r) for r in rew_raw]

In [10]:
rew

[2.8083882658399464,
 3.0847240625277044,
 3.117140228076572,
 2.9460630169150575,
 1.2853387653114443,
 1.417470355233744,
 1.357721420027428,
 0.44462066539922196,
 1.4359515648184173,
 2.532295799086351,
 2.638181412481392,
 2.9879326107097786,
 0.20499089791295289,
 -0.23181387447790036,
 0.6451462100843955,
 1.7068034459173957,
 2.532295799086351,
 1.8012898000172166,
 1.7686705302397543,
 2.532295799086351,
 2.6890835596657072,
 0.3839541916730141,
 0.5024454604782219,
 0.70335965818934]

In [17]:
import torch
a = torch.Tensor([[1,1],[2,2]])
b = torch.Tensor([[2,2],[2,2]])
torch.nn.PairwiseDistance(p=2)(a.reshape(-1),b.reshape(-1))

tensor(1.4142)

In [18]:
from transformers import T5ForConditionalGeneration

[2024-01-08 16:04:21,903] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [19]:
model = T5ForConditionalGeneration.from_pretrained('./tasks/qa_feedback/model_outputs/t5-large-1k-train')

In [41]:
model.decoder

T5Stack(
  (embed_tokens): Embedding(32100, 1024)
  (block): ModuleList(
    (0): T5Block(
      (layer): ModuleList(
        (0): T5LayerSelfAttention(
          (SelfAttention): T5Attention(
            (q): Linear(in_features=1024, out_features=1024, bias=False)
            (k): Linear(in_features=1024, out_features=1024, bias=False)
            (v): Linear(in_features=1024, out_features=1024, bias=False)
            (o): Linear(in_features=1024, out_features=1024, bias=False)
            (relative_attention_bias): Embedding(32, 16)
          )
          (layer_norm): T5LayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): T5LayerCrossAttention(
          (EncDecAttention): T5Attention(
            (q): Linear(in_features=1024, out_features=1024, bias=False)
            (k): Linear(in_features=1024, out_features=1024, bias=False)
            (v): Linear(in_features=1024, out_features=1024, bias=False)
            (o): Linear(in_features=1024, out_feat

In [40]:
torch.all(model.decoder.block[-1].layer[0].SelfAttention.q.weight==model.decoder.block[-2].layer[0].SelfAttention.q.weight)

tensor(True)

In [ ]:
model1 = AutoModel.from_pretrained('t5-base')
model2 = T5ForConditionalGeneration.from_pretrained('t5-base')
o1 = model1(torch.tensor([[351]*10,[351]*10,[351]*10]),decoder_input_ids=torch.tensor([[156]*10,[351]*10,[351]*10]))
o2 = model2(torch.tensor([[350]*10,[352]*10,[353]*10]),decoder_input_ids=torch.tensor([[150]*10,[351]*10,[768]*10]),output_hidden_states=True)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')

In [ ]:
inp = tokenizer(['Hello, how are you?','There was a village nearby'], return_tensors='pt')

In [ ]:
out = model2.generate(input_ids=inp.input_ids, num_return_sequences=4, do_sample=True)

In [ ]:
tokenizer.batch_decode(out)

In [ ]:
o2.decoder_hidden_states[-1].shape

In [ ]:
l = torch.nn.Linear(768,5)
lo = l(o2.decoder_hidden_states[-1].permute(0,1,2))

In [ ]:
lo

In [ ]:
lo.permute(2,0,1)

In [ ]:
t = torch.tensor([[[1,2,3,4,5],[6,7,8,9,0]],[[10,20,30,40,50],[60,70,80,90,110]],[[100,200,300,400,500],[600,700,800,900,1100]]])
t

In [ ]:
i = torch.tensor([1,2])
torch.gather(t,2,i.repeat(3,1)[:,:,None])

In [ ]:
i.repeat(3,1)[:,:,None]

In [ ]:
torch.distributions.Categorical(t).entropy()

In [ ]:
t[0,0]

In [ ]:
t.repeat(1,3).reshape(2,-1,5)

In [ ]:
init_wts = model2.lm_head.weight.detach()
init_wts.shape

In [ ]:
model2.lm_head = torch.nn.Linear(in_features=768,out_features=32128*3, bias=False)
model2.lm_head.weight

In [ ]:
model2.lm_head.weight = torch.nn.Parameter(init_wts.repeat(3,1))
model2.lm_head.weight

In [ ]:
o = model2(input_ids=torch.tensor([[0,1,2,3,4,5],[0,1,2,3,4,5]]),decoder_input_ids=torch.tensor([[2,4,5,6,3],[2,4,5,6,3]]),output_hidden_states=True)

In [ ]:
o.decoder_hidden_states[-1].shape

In [ ]:
B,SL,V = o.logits.shape
SL

In [ ]:
o.logits.reshape(B,SL,3,-1).permute(2,0,1,3)

In [ ]:
t = torch.tensor([[1,2,3,4,5],[6,7,8,9,0]])
t.index_select(-1,torch.tensor([0]))

In [ ]:
del(model2.lm_head)

In [ ]:
model2

In [ ]:
model2.config

In [ ]:
t = torch.tensor([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])
t.shape

In [ ]:
t.reshape(2,2,3)

In [ ]:
model2.lm_head.weight

In [ ]:
model2.lm_head.weight = torch.nn.Parameter(torch.clone(model2.lm_head.weight))

In [ ]:
t1 = torch.tensor([[1,2,3],[4,5,6]])
t2 = torch.tensor([[7,8,9],[0,1,2]])
torch.concat([t1.unsqueeze(0),t2.unsqueeze(0)],dim=0).shape

In [ ]:
prompts_text = ['Hello, how are you?','There was a village nearby']
[elem for elem in prompts_text for _ in range(4)]*2

In [ ]:
LL = torch.nn.Linear(in_features=768,out_features=4)

In [ ]:
op=LL(o2.decoder_hidden_states[-1])
len = op.shape[1]
op.permute(2,0,1).reshape(-1,len)

In [ ]:
l = [1,2,3]
l*3

In [ ]:
t1[0].shape

In [ ]:
d = {'a':1, 'b':2, 'c':3}
d

In [ ]:
d.update({'d':4})
d

In [ ]:
del(d['d'])

In [ ]:
d

In [ ]:
t1 = torch.tensor([1,2,3])
t2 = torch.tensor([4,5,6])
torch.concat([t1,t2])